<h1> Execution of Groamscs on Unix System using NVIDIA GROMACS </h1>

<h1> Requirements </h1>

#Note: These commands shall be executed on - GPU based system. If that has Jupyter support, then you can run this on 
#Jupyter Notebook. Otherwise use this commands on Shell Prompt on Unix / Linux based systems.

In [58]:
# GROMACS REQUIRES.
# NVIDIA GPU or other GPU
# CUDA 10.1+
# NVIDIA Docker of GROMACS  from ,  NGCngc.nvidia.com › catalog › containers › hpc:gromacs : search NVIDIA gromacs docker
# If you are using above, then install docker, nvidia-docker (docker2) module

## OR
## You can install GROMACS directly, without using NVIDIA gromacs container - docker


# If using NVIDIA  - Then follow this synatx. Data22 is my folder name. you can use yours !!!
# sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx 

# if Gromacs directly without NVIDIA Gromacs, then use.
# gmx 

In [59]:
# Please study and execute GROMACS using http://www.mdtutorials.com/gmx/. Please study before, executing  below commands.
# Study Tutorial 1 and Tutorial 5.

<h3> Prepare Protein </h3

In [52]:
# You have to combine protein and ligand in one file using PyMOL software.
# after saving again open and see that complex, and observe protein and ligand placement
# This Protein is not good for Gromacs
# You have to download swisspdb viewer. once downloaded, upload your complex protein, swisspdb viewer 
# will fix missing Hydrogens, residues and other abnormalities.
# After corrected pdb, you have to save as "Final PDB" for your start.

<h3>Prepare Ligand</h3>

In [54]:
# Your Ligand preparation
# Using any text editor, you can remove HETATOMS from above complex.pdb, and save in other file as Ligand.pdb
# Now open with Pymol and add Hydrogens and save as ligand.mol2
# This ligand.mol2 has to uplaod to SWISSPARAM.CH website, and  you will get zip file, after few minutes, If all well,
# then, unzip that folder, you have to get ligand.itp, ligand.prm, ligand.pdb
# copy these 3 files, Yes ligand.pdb also, which will replace your old ligand.pdb


<h1> Start Gromacs </h1>

In [ ]:
 ### Execute below commands on Shell in Unix / Linux #######################

In [53]:
#sudo docker run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs gmx pdb2gmx -f complex_swiss_x.pdb -ff charmm36-mar2019 -ignh -o complex_full_swiss_processed.gro -nochargegrp
# select  5 - SPE

In [55]:
# if any thing wrong, please check your file of protein, complex, and repeat above steps !!

In [3]:
#sudo docker run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs gmx editconf -f ligand.pdb -o ligand.gro

In [4]:
# Combine "GRO" file of Protein and Ligand into proyein.gro
# Chnage shall be done in Topol.top

In [5]:
#sudo docker run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs gmx editconf -f complex_full_swiss_processed.gro -o boxed.gro -c -d 1.0 -bt cubic


In [6]:
#sudo docker run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs  gmx solvate -cp boxed.gro -cs spc216.gro -p topol.top -o solv.gro

In [7]:
#sudo docker run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs gmx grompp -f mdp/ions.mdp -c solv.gro -p topol.top -o ions.tpr 
# you may get problem, if properly you have not created files of proteins and ligands

In [ ]:
#sudo docker run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -pname NA -nname CL -neutra

In [12]:
# sudo docker run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs gmx editconf -f boxed.gro -o boxed.pdb
# at this level, please check your protein-ligand complex. if your ligand is farway, or which is diffrent 
# than docking structure, then it is wrong. first correct that, and then proceed from first step again

In [10]:
#sudo docker run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx grompp -f mdp/em.mdp -c solv_ions.gro -p topol.top -o em.tpr
# em.mdp, please checkin MDP folder

In [11]:
# sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs     gmx mdrun -v -deffnm em -nb gpu  
# GPU should be selected automatically, if youhave installed NVIDIA and CUDA drivers.

In [13]:
# NVT
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs  gmx make_ndx -f ligand.pdb -o index_ligand.ndx

#> 0  & ! a H*
#> q

In [14]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs  gmx genrestr -f ligand.pdb -n index_ligand.ndx -o posre_ligand.itp -fc 1000 1000 1000
#> 3

In [15]:
#in Topol.top

#== in topol.top
#; Ligand position restraints
#ifdef POSRES_LIG
#include "posre_ligand.itp"
#endif
#====

In [16]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx make_ndx -f em.gro -o index.ndx
#> 1 | 13
#> q

In [17]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx grompp -f mdp/nvt.mdp -c em.gro -r em.gro -p topol.top -n index.ndx -o nvt.tpr

In [18]:
# sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx mdrun -deffnm nvt -nb gpu
# this step will take more than hour or so, depend on your system.

In [19]:
# NPT
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx grompp -f mdp/npt.mdp -c nvt.gro -t nvt.cpt -r nvt.gro -p topol.top -n index.ndx -o npt.tpr

In [20]:
# sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx mdrun -deffnm npt -nb gpu

In [21]:
# MOLECULAR DYNAMICS - ACTUAL Computation !!

In [22]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx grompp -f mdp/md20ns.mdp -c npt.gro -t npt.cpt -p topol.top -n index.ndx -o md20ns.tpr

<h3>Molecular Dynamics </h3>

In [24]:
# sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs  gmx mdrun -deffnm md20ns -nb gpu  -pme gpu  -bonded gpu

# THIs STEP WILL TAKE 24 HOURS ON nvidia 1080, 22 gb ram, 2 GPU.
# If you are using laptop,with 8 GB RAM only, then it may take 10 Days and even more.

<h1> After GROMACS MD SIMULATION </h1>

In [25]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx trjconv -s md20ns.tpr -f md20ns.xtc  -center -pbc mol -ur compact -o md20ns_center.xtc
# "Protein\nSystem\n

In [26]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx trjconv -s md20ns.tpr -f md20ns_center.xtc -fit rot+trans -o md20ns_fit.xtc 
#"Backbone\nSystem\n

In [27]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx trjconv -s md20ns.tpr -f md20ns_fit.xtc -dump 2000  -o start2000.pdb
#"protein" - Last frame

In [28]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx make_ndx -f em.gro -n index.ndx
#...
# > select protein Group number
# (creates group 23)
#> select Ligand Group number
# (creates group 24)
# > 23 | 24
# > q

In [29]:
# Potential Energy
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx energy -f em.edr -o potentialEnergy.xvg
#select number of "Potential" and "system" 
# please check the number in your version of Gromacs

In [30]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx energy -f nvt.edr -o temperature.xvg
# temperature, system

In [31]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx energy -f npt.edr -o density.xvg
# Density, system

In [32]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx energy -f npt.edr -o pressure.xvg
#pressure, system

In [33]:
# If you want, no PBC
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx trjconv -s md20ns.tpr -f md20ns.xtc -o md20ns_noPBC.xtc -pbc mol -center

#Protein , System

In [38]:
 # RMSD - Calpha

# sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx  rms -f md20ns_protein.xtc -s md20ns.pdb -o rmsd_pdb_CA.xvg -n NC_distance.ndx -tu ns
 
# C ALPHA, Protein


In [37]:
# RMSD Backbone
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx  rms -f md20ns_protein.xtc -s md20ns.pdb -o rmsd_pdb_backbone.xvg -n NC_distance.ndx -tu ns
# Backbone,system 
    

In [36]:
# Interaction Energy
# sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx grompp -f mdp/ie.mdp -c npt.gro -t npt.cpt -p topol.top -n index.ndx -o ie.tpr
# sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx mdrun -deffnm ie -rerun md20ns_fit.xtc -nb cpu
# 
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx energy -f ie.edr -o interaction_energy_20ns.xvg
    

In [39]:
# RAMACHNADRAN Plot
# sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs  gmx rama -f md20ns.pdb -s md20ns.tpr -o rama_protein_0.xvg

# You need GNUPLOT for windows, XMGRACE on Linux to view XVG file
# on gnuplot> plot("c:\\..\\rama_protein_0.xvg")

In [40]:
# Hydrogen Bonds
# HBond
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx make_ndx -f md20ns.tpr -o index.ndx
#1 , 19, 22 | 23, q - selecting this depends on what you want. Protein - Ligand, Protein-protein.
# depend on your work choose correct numbers, when prompted

In [41]:
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs    gmx hbond -f md20ns.xtc -s md20ns.tpr -num hbnum.xvg -g hbond.log -ac hbac.xvg -dist hbdist.xvg -ang hbang.xvg -hx hbhelix.xvg  -hbn index.ndx -hbm hbmap.xpm -don donor.xvg -dan danum.xvg -life hblife.xvg -nhbdist nhbdist.xvg 

# Protein, Ligand  ( or) Protein, protein or your choice as per your experiment

In [42]:
# Cluster
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs    gmx cluster -f md20ns_protein.xtc -s md20ns_protein.tpr  -method gromos -om rmsd-raw.xpm -o rmsd-clust.xpm -g cluster.log -dist rmsd-dist.xvg -ev rmsd-eig.xvg -conv mc-conv.xvg -sz clust-size.xvg -ntr clust-trans.xvg -clid clust-id.xvg -cl clusters.pdb -tu ns  -cutoff 0.15
# get clusters for  CAlpha and Protein or you can choose your choice
    

In [44]:
# REMOVE WATER - Because output file i very big !!
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx  trjconv -f md20ns.xtc -s md20ns.tpr -o prd-lig.xtc -center -ur compact -pbc mol

#Proten-Ligand  or
#Clapha - Ligand

# this is good for VMD  visualization.

In [45]:
# dssp
# I am unable to execute code, but used online services like   http://endscript.ibcp.fr/ESPript/ENDscript/
# upload your last frame PDB from below command.
# sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx trjconv -s md20ns.tpr -f md20ns_fit.xtc -dump 2000  -o start2000.pdb

#

In [46]:
#   Gyration
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx  gyrate -f md20ns_protein.xtc -s md20ns.pdb -o gyrate_pdb.xvg
# Rgyr is a measure for the compactness of a structure

In [47]:
# RMSF
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx rmsf -f md20ns_protein.xtc -s md20ns.pdb -o rmsf_pdb.xvg
    

In [49]:

# Distance 
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx distance -s md20ns.tpr -f md20ns_fit.xtc -select -oall
# OR 
#sudo nvidia-docker  run -v $HOME/data22:/data22 -w /data22 -it gromacs/gromacs   gmx distance -s md20ns.tpr -f md20ns_fit.xtc  -oall md20ns_dist.xvg -oh md20ns_disthist.xvg -n index.ndx -tu ns -len 2 -binw 0.01

# # select your choice, when prompted
    
    